Processing the open food databse to extract a dataset to use for the visualization.

In [1]:
import pandas as pd
import numpy as np
import re
from scipy import sparse as sparse

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

data_dir = "/Users/seddont/Dropbox/Tom/MIDS/W209_work/Tom_project/"
code_dir = "/Users/seddont/Dropbox/Tom/MIDS/W209_work/w209finalproject/"

Working from the full database, because the usda_imports_filtered.csv file in the shared drive does not have brand information, which might be useful for displaying.

In [2]:
# Get sample of the full database to understand what columns we want
smp = pd.read_csv(data_dir+"en.openfoodfacts.org.products.csv", sep = "\t", nrows = 100)

In [3]:
for c in smp.columns:
    print(c)

code
url
creator
created_t
created_datetime
last_modified_t
last_modified_datetime
product_name
generic_name
quantity
packaging
packaging_tags
brands
brands_tags
categories
categories_tags
categories_en
origins
origins_tags
manufacturing_places
manufacturing_places_tags
labels
labels_tags
labels_en
emb_codes
emb_codes_tags
first_packaging_code_geo
cities
cities_tags
purchase_places
stores
countries
countries_tags
countries_en
ingredients_text
allergens
allergens_en
traces
traces_tags
traces_en
serving_size
no_nutriments
additives_n
additives
additives_tags
additives_en
ingredients_from_palm_oil_n
ingredients_from_palm_oil
ingredients_from_palm_oil_tags
ingredients_that_may_be_from_palm_oil_n
ingredients_that_may_be_from_palm_oil
ingredients_that_may_be_from_palm_oil_tags
nutrition_grade_uk
nutrition_grade_fr
pnns_groups_1
pnns_groups_2
states
states_tags
states_en
main_category
main_category_en
image_url
image_small_url
energy_100g
energy-from-fat_100g
fat_100g
saturated-fat_100g
-buty

In [4]:
# Specify what columns we need for the visualization.  For speed purposes going to
# remove any we don't really need

wanted_cols = ['code', 'creator', 'product_name', 'brands', 'brands_tags', 'serving_size',
       'serving_size', 'energy_100g', 'fat_100g', 'cholesterol_100g',
       'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']

# Create a list of columns to drop to check it worked ok
drop_cols = [c for c in smp.columns if c not in wanted_cols]
print(drop_cols)

['url', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags', 'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil', 'ingredients_from_palm_oil_tags', 'ingredients_that_may_be_from_palm_oil_n', 'ingredients_that_may_be_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_tags', 'nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en', 'main_category', 'mai

In [5]:
# Pull in full dataset, only the columns we want
df = pd.read_csv(data_dir+"en.openfoodfacts.org.products.csv", sep = "\t")

/Users/seddont/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Drop unwanted columns
df.drop(drop_cols, axis = 1, inplace = True)

In [7]:
# Take a quick look
df

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g
0,3087,openfoodfacts-contributors,Farine de blé noir,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,usda-ndb-import,Banana Chips Sweetened (Whole),NaN,NaN,28 g (1 ONZ),2243.0,28.57,0.018,64.29,14.29,3.6,3.57,0.000000
2,4559,usda-ndb-import,Peanuts,Torn & Glasser,torn-glasser,28 g (0.25 cup),1941.0,17.86,0.000,60.71,17.86,7.1,17.86,0.250000
3,16087,usda-ndb-import,Organic Salted Nut Mix,Grizzlies,grizzlies,28 g (0.25 cup),2540.0,57.14,NaN,17.86,3.57,7.1,17.86,0.482000
4,16094,usda-ndb-import,Organic Polenta,Bob's Red Mill,bob-s-red-mill,35 g (0.25 cup),1552.0,1.43,NaN,77.14,NaN,5.7,8.57,NaN
5,16100,usda-ndb-import,Breadshop Honey Gone Nuts Granola,Unfi,unfi,52 g (0.5 cup),1933.0,18.27,NaN,63.46,11.54,7.7,13.46,NaN
6,16117,usda-ndb-import,Organic Long Grain White Rice,Lundberg,lundberg,45 g (0.25 cup),1490.0,NaN,NaN,80.00,NaN,NaN,8.89,NaN
7,16124,usda-ndb-import,Organic Muesli,Daddy's Muesli,daddy-s-muesli,64 g (0.5 cup),1833.0,18.75,NaN,57.81,15.62,9.4,14.06,0.055000
8,16193,usda-ndb-import,Organic Dark Chocolate Minis,Equal Exchange,equal-exchange,40 g (40 g),2406.0,37.50,NaN,55.00,42.50,7.5,5.00,NaN
9,16513,usda-ndb-import,Organic Sunflower Oil,Napa Valley Naturals,napa-valley-naturals,14 g (1 Tbsp),3586.0,100.00,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop all rows that are not from the usda ndb import
df = df[df.creator == "usda-ndb-import"]

# Drop all rows where Brands == Nan as we can't really identify those products
df = df[df.brands.notnull()]

In [9]:
df

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g
2,4559,usda-ndb-import,Peanuts,Torn & Glasser,torn-glasser,28 g (0.25 cup),1941.0,17.86,0.000,60.71,17.86,7.1,17.86,0.250
3,16087,usda-ndb-import,Organic Salted Nut Mix,Grizzlies,grizzlies,28 g (0.25 cup),2540.0,57.14,NaN,17.86,3.57,7.1,17.86,0.482
4,16094,usda-ndb-import,Organic Polenta,Bob's Red Mill,bob-s-red-mill,35 g (0.25 cup),1552.0,1.43,NaN,77.14,NaN,5.7,8.57,NaN
5,16100,usda-ndb-import,Breadshop Honey Gone Nuts Granola,Unfi,unfi,52 g (0.5 cup),1933.0,18.27,NaN,63.46,11.54,7.7,13.46,NaN
6,16117,usda-ndb-import,Organic Long Grain White Rice,Lundberg,lundberg,45 g (0.25 cup),1490.0,NaN,NaN,80.00,NaN,NaN,8.89,NaN
7,16124,usda-ndb-import,Organic Muesli,Daddy's Muesli,daddy-s-muesli,64 g (0.5 cup),1833.0,18.75,NaN,57.81,15.62,9.4,14.06,0.055
8,16193,usda-ndb-import,Organic Dark Chocolate Minis,Equal Exchange,equal-exchange,40 g (40 g),2406.0,37.50,NaN,55.00,42.50,7.5,5.00,NaN
9,16513,usda-ndb-import,Organic Sunflower Oil,Napa Valley Naturals,napa-valley-naturals,14 g (1 Tbsp),3586.0,100.00,NaN,NaN,NaN,NaN,NaN,NaN
10,16612,usda-ndb-import,Organic Adzuki Beans,Unfi,unfi,48 g (0.25 cup),1393.0,1.04,NaN,62.50,NaN,12.5,22.92,NaN
11,16650,usda-ndb-import,Organic Penne Pasta,Gardentime,gardentime,57 g (0.5 cup),1540.0,1.75,NaN,73.68,NaN,1.8,14.04,NaN


The code column can be problematic as it's a long number that can be truncated to a floating point representation when manipulated by certain programs.  Going to convert it to append a character at the start to it will be read unambiguously as a string.

First convert the code column to a string

In [10]:
df.code.apply(str)

2                  4559
3                 16087
4                 16094
5                 16100
6                 16117
7                 16124
8                 16193
9                 16513
10                16612
11                16650
12                16872
13                16933
14                17497
15                18012
16                18050
17                18173
18                18197
19                18227
20                18265
21                18289
22                18319
23                18340
24                18357
25                18371
26                18388
27                18395
28                18401
29                18418
30                18449
31                18456
              ...      
330245    9556041600040
330246    9556041600057
330248    9556041600255
330249    9556041600293
330250    9556041602143
330251    9556041602211
330258    9556041608947
330259    9556041608954
330266    9556041610940
330273    9556041612593
330274    955604

In [11]:
# Add an N in front of the number string
df.code = "N"+df.code.astype(str)

Quick check for NA values

In [12]:
df.isnull().sum(axis = 0)

code                      0
creator                   0
product_name             45
brands                    0
brands_tags               0
serving_size              2
energy_100g             396
fat_100g                399
cholesterol_100g      28029
carbohydrates_100g      273
sugars_100g           11314
fiber_100g            31282
proteins_100g           891
sodium_100g             790
dtype: int64

Need to get rid of rows with null product_name as we will be using that later for display etc.

In [13]:
df = df[df.product_name.notnull()]

Now going to want to try and find similar products to a specific item.  In the demo, did this by just pulling items with certain text in the name and then doing some hand sorting.  Can't scale that approach.  Going to try to do it based on a combination of words in the name and the ingredients.

Try out the approach of using CountVectorizer on the product_name field to see how it does.

In [14]:
df

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g
2,N4559,usda-ndb-import,Peanuts,Torn & Glasser,torn-glasser,28 g (0.25 cup),1941.0,17.86,0.000,60.71,17.86,7.1,17.86,0.250
3,N16087,usda-ndb-import,Organic Salted Nut Mix,Grizzlies,grizzlies,28 g (0.25 cup),2540.0,57.14,NaN,17.86,3.57,7.1,17.86,0.482
4,N16094,usda-ndb-import,Organic Polenta,Bob's Red Mill,bob-s-red-mill,35 g (0.25 cup),1552.0,1.43,NaN,77.14,NaN,5.7,8.57,NaN
5,N16100,usda-ndb-import,Breadshop Honey Gone Nuts Granola,Unfi,unfi,52 g (0.5 cup),1933.0,18.27,NaN,63.46,11.54,7.7,13.46,NaN
6,N16117,usda-ndb-import,Organic Long Grain White Rice,Lundberg,lundberg,45 g (0.25 cup),1490.0,NaN,NaN,80.00,NaN,NaN,8.89,NaN
7,N16124,usda-ndb-import,Organic Muesli,Daddy's Muesli,daddy-s-muesli,64 g (0.5 cup),1833.0,18.75,NaN,57.81,15.62,9.4,14.06,0.055
8,N16193,usda-ndb-import,Organic Dark Chocolate Minis,Equal Exchange,equal-exchange,40 g (40 g),2406.0,37.50,NaN,55.00,42.50,7.5,5.00,NaN
9,N16513,usda-ndb-import,Organic Sunflower Oil,Napa Valley Naturals,napa-valley-naturals,14 g (1 Tbsp),3586.0,100.00,NaN,NaN,NaN,NaN,NaN,NaN
10,N16612,usda-ndb-import,Organic Adzuki Beans,Unfi,unfi,48 g (0.25 cup),1393.0,1.04,NaN,62.50,NaN,12.5,22.92,NaN
11,N16650,usda-ndb-import,Organic Penne Pasta,Gardentime,gardentime,57 g (0.5 cup),1540.0,1.75,NaN,73.68,NaN,1.8,14.04,NaN


In [15]:
# Get all the non-null product name fields
name_data = df.product_name

# Pass them to the Count Vectorizer
vectorizer = CountVectorizer()
wv = vectorizer.fit_transform(name_data)

# Get some basic stats
print("Size of vocabulary:", wv.shape[1],"words")
print("Average non-zero entries per example:%5.2f" % (1.0*wv.nnz/wv.shape[0]))

Size of vocabulary: 19299 words
Average non-zero entries per example: 3.99


Get the extracted words

In [16]:
a = vectorizer.get_feature_names()
print("First feature string:", sorted(a)[0])
print("Last feature string:", sorted(a)[len(a)-1])

First feature string: 00
Last feature string: zymbom


Count occurences by word

In [17]:
fn = vectorizer.get_feature_names()
wc = wv.sum(axis = 0)
word_frame = pd.DataFrame({"word": fn, "count":np.ravel(wc)}) 

In [18]:
word_frame.sort_values(by = ["count"], ascending = False)

,count,word
3523,10278,chocolate
3247,9972,cheese
12211,9052,organic
18888,7899,with
15012,7646,sauce
11097,6765,mix
4348,5740,cream
10976,4825,milk
8889,4200,juice
6662,4178,fruit


Ok -- challenge here is that many of the words are descriptive adjectives vs nouns.  Even the ones that are nouns are going to be hard to separate e.g. rice could be rice crackers, chicken and rice, etc.

Wonder if I can get anywhere with bigrams or trigrams...try again

In [19]:
# Get all the non-null product name fields
name_data = df.product_name[df.product_name.notnull()]

# Pass them to the Count Vectorizer
vectorizer = CountVectorizer(analyzer = "word", ngram_range = (3,3))
wv = vectorizer.fit_transform(name_data)

# Get some basic stats
print("Size of vocabulary:", wv.shape[1],"words")
print("Average non-zero entries per example:%5.2f" % (1.0*wv.nnz/wv.shape[0]))

fn = vectorizer.get_feature_names()
wc = wv.sum(axis = 0)

word_frame = pd.DataFrame({"word": fn, "count":np.ravel(wc)}) 

word_frame.sort_values(by = ["count"], ascending = False)

Size of vocabulary: 192887 words
Average non-zero entries per example: 2.09


,count,word
182900,810,virgin olive oil
60742,807,extra virgin olive
59799,750,enriched macaroni product
134808,558,premium ice cream
152804,367,sharp cheddar cheese
79494,358,greek nonfat yogurt
42253,303,cooked potato chips
160718,296,sparkling water beverage
88910,280,in light syrup
142163,268,reduced fat milk


So some kind of bigram and trigram approach might be scalable here.  But don't really have a lot of time to perfect that.  However, could use these trigrams to expand the simple 'demo cat' approach by picking only certain bigrams and then using the simple algorithm from the demo approach to find a range of other products with that bigram.

Going to write out the trigrams, do some hand coding of which ones we want to use, then bring the results back in

In [20]:
word_frame.to_csv(data_dir+"trigrams_uncoded.csv")

In [81]:
tri_w = pd.read_csv(code_dir+"trigrams_wanted_v2.csv")

In [82]:
tri_w

,count,word,wanted
0,811,virgin olive oil,NaN
1,808,extra virgin olive,NaN
2,751,enriched macaroni product,NaN
3,558,premium ice cream,1.0
4,369,sharp cheddar cheese,NaN
5,358,greek nonfat yogurt,1.0
6,303,cooked potato chips,1.0
7,296,sparkling water beverage,NaN
8,285,in light syrup,NaN
9,268,reduced fat milk,NaN


Can drop all the rows where wanted is NaN

In [83]:
tri_w = tri_w[tri_w.wanted.notnull()]
tri_w

,count,word,wanted
3,558,premium ice cream,1.0
5,358,greek nonfat yogurt,1.0
6,303,cooked potato chips,1.0
14,223,cream cheese spread,1.0
20,197,red kidney beans,1.0
25,174,frosted sugar cookies,1.0
26,173,chunk light tuna,1.0
39,144,roasted turkey breast,1.0
40,142,organic baby food,1.0
42,136,chocolate chip cookie,1.0


So now got an identifier to use to append wanted categories from

Want to append the trigram as the 'demo category' for any product which contains that trigram.  It is possible that a product will fit into multiple trigrams, in which case I will choose to put it in the lowest total count trigram category.  Going to do this by looping through the wanted trigrams from highest to lowest and updating the product's category as needed.

In [84]:
# Initialize demo category with None
df["demo_cat"] = "None"

# loop over each trigram
for trigram in tri_w.word:
 
    # get the index of the correct column for that trigram in the vectorized output
    wv_index = fn.index(trigram)
    
    # Get locations of matches and convert to a dense representation for indexing
    matches = wv[:,wv_index] == 1    
    matches = np.ravel(sparse.csr_matrix.todense(matches))
      
    # Set the 'demo_cat' field to that trigram value
    df.loc[matches, ["demo_cat"]] = trigram

/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/seddont/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [85]:
df[df.demo_cat != "None"]

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat
39,N18708,usda-ndb-import,Organic Black Beans,Unfi,unfi,45 g (0.25 cup),1393.0,NaN,NaN,62.22,2.22,24.4,22.22,NaN,organic black beans
51,N30557,usda-ndb-import,Organic Green Split Peas,Coombs Family,coombs-family,45 g (0.25 cup),1582.0,1.11,NaN,68.89,NaN,28.9,28.89,0.022,green split peas
130,N38713,usda-ndb-import,Organic Steel-Cut Oats,Unfi,unfi,40 g (0.25 cup),1778.0,7.50,NaN,72.50,NaN,12.5,17.50,NaN,steel cut oats
139,N39536,usda-ndb-import,Organic Short Grain Brown Rice,Lundberg,lundberg,51 g (0.25 cup),1393.0,2.94,NaN,78.43,NaN,5.9,5.88,NaN,grain brown rice
292,N433821494,usda-ndb-import,Oatmeal Raisin Cookies,Bart & Judy's,bart-judy-s,28 g (6 COOKIES),1552.0,21.43,0.036,67.86,28.57,3.6,7.14,0.214,oatmeal raisin cookies
417,N901000017,usda-ndb-import,Yellow Corn Tortilla Chips,Guiltless Gourmet,guiltless-gourmet,28 g (1 oz),1795.0,10.71,0.000,78.57,0.00,7.1,7.14,0.643,yellow corn tortilla
483,N16056,usda-ndb-import,Whole Kernel Corn,Trader Joe's,trader-joe-s,125 g (0.5 cup),201.0,1.20,0.000,7.20,5.60,1.6,1.60,0.160,whole kernel corn
671,N70662455029,usda-ndb-import,Top Ramen Noodle Soup,Nissin,nissin,42 g (0.5 DRY NOODLE BLOCK AND 1 TSP SEASONING...,1891.0,16.67,0.000,64.29,0.00,4.8,11.90,1.810,ramen noodle soup
718,N79969,usda-ndb-import,Dry Roasted & Salted Almonds,Trader Joe's,trader-joe-s,30 g (0.25 cup),2372.0,50.00,0.000,16.67,3.33,13.3,23.33,0.383,roasted salted almonds
720,N8001000661,usda-ndb-import,"Cool Beans, Red Pepper Hummus",Bobbi's Best Inc.,bobbi-s-best-inc,28 g (2 Tbsp),895.0,21.43,0.000,10.71,0.00,3.6,3.57,0.375,red pepper hummus


Some categories have very little variation in them.  Going to check for that and drop them.

In [86]:
# loop over each trigram
enough_variance = []

for trigram in tri_w.word:
    
    fat_var = df[df.demo_cat == trigram]["fat_100g"].var()
    
    if fat_var > 2:
        enough_variance.append(trigram)

print(enough_variance)
print(len(enough_variance))

['premium ice cream', 'cooked potato chips', 'cream cheese spread', 'frosted sugar cookies', 'chunk light tuna', 'roasted turkey breast', 'chocolate chip cookie', 'hot dog buns', 'ice cream sandwiches', 'cookies chocolate chip', 'ice cream bars', 'crunchy peanut butter', 'pancake waffle mix', 'chewy granola bars', 'dry roasted peanuts', 'dairy frozen dessert', 'thin crust pizza', 'macaroni cheese dinner', 'vegetable oil spread', 'chili with beans', 'chicken noodle soup', 'peanut butter cups', 'albacore tuna in', 'dark chocolate bar', 'potato chips original', 'honey roasted peanuts', 'roasted salted almonds', 'ramen noodle soup', 'fat greek yogurt', 'whole grain bread', 'smoked turkey breast', 'marinara pasta sauce', 'wavy potato chips', 'dark chocolate almonds', 'peanut butter cup', 'high protein bar', 'french fried potatoes', 'organic pasta sauce', 'sardines in tomato', 'italian style meatballs', 'fruit nut bar', 'thousand island dressing', 'baked snack crackers', 'cookies peanut butt

Now got a category applied to a subset of the database.  Can run the same code as before to use that category to create a subset of recommendations for each category picked...

In [87]:
# What we want to get variation on
pick_factors = ['fat_100g', 'sugars_100g', 'proteins_100g', 'sodium_100g']

# Points we want to pick (percentiles).  Can tune this to get more or fewer picks.

pick_percentiles = [0.1, 0.5, 0.9]
# pick_percentiles = [0, 0.25, 0.5, 0.75, 1.0]

demo_picks = []

# loop over each trigram that has enough variance in it

for cat in enough_variance:
    
    # first get all the items containing the cat word
    catf = df[df["demo_cat"] == cat]
        
    # Identify what rank each product is in that category, for each main factor
    for p in pick_factors:
        
        catf[p + "_rank"] = catf[p].rank(method = "first")
        
        # Select products at chosen percentiles on each
        
        high = catf[p + "_rank"].max()
        
        pick_index = [max(1, round(n * high)) for n in pick_percentiles]
        
        # add codes for those products
        demo_picks.extend(catf[catf[p+"_rank"].isin(pick_index)].code)
        
    
demo_df = df[df.code.isin(demo_picks)]
demo_df

/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat
417,N901000017,usda-ndb-import,Yellow Corn Tortilla Chips,Guiltless Gourmet,guiltless-gourmet,28 g (1 oz),1795.0,10.71,0.000,78.57,0.00,7.1,7.14,0.643,yellow corn tortilla
1366,N98601,usda-ndb-import,Chicken Noodle Soup,Trader Joe's,trader-joe-s,248 g (1 cup),151.0,0.40,0.008,5.65,0.40,0.4,2.42,0.294,chicken noodle soup
2267,N11110197900,usda-ndb-import,"Triple Layer Red Velvet Cake, Peppermint","Bakery Fresh Goodness, The Kroger Co.","bakery-fresh-goodness,the-kroger-co",85 g (1 SLICE),1971.0,17.65,0.024,77.65,65.88,0.0,1.18,0.353,red velvet cake
2669,N11110577276,usda-ndb-import,Ice Cream Sandwich,The Kroger Co.,the-kroger-co,63 g (1 SANDWICH),1130.0,9.52,0.032,44.44,20.63,1.6,4.76,0.238,ice cream sandwich
3172,N11110791559,usda-ndb-import,Crunchy Peanut Butter,Simple Truth Organic,simple-truth-organic,32 g (2 Tbsp),2615.0,50.00,0.000,18.75,6.25,9.4,25.00,0.375,crunchy peanut butter
3187,N11110796486,usda-ndb-import,"Kettle Cooked Potato Chips, Sea Salt & Cracked...",Kroger,kroger,28 g (1 oz),2243.0,32.14,0.000,53.57,0.00,3.6,7.14,0.393,cooked potato chips
3427,N11110819376,usda-ndb-import,Dry Roasted Peanuts,Kroger,kroger,28 g (1 oz),2389.0,46.43,0.000,21.43,3.57,7.1,21.43,0.607,dry roasted peanuts
3638,N11110836281,usda-ndb-import,53% Vegetable Oil Spread,P$$T... Big Savings... Pass It On,p-t-big-savings-pass-it-on,14 g (1 Tbsp),2092.0,50.00,0.000,0.00,0.00,0.0,0.00,0.643,vegetable oil spread
3654,N11110837158,usda-ndb-import,98% Fat Free Cream Of Mushroom Condensed Soup,Kroger,kroger,125 g (0.5 cup),268.0,2.00,0.004,8.80,1.60,0.0,1.60,0.640,mushroom condensed soup
3701,N11110839305,usda-ndb-import,Chili With Beans,Kroger,kroger,250 g (1 cup),502.0,4.00,0.010,14.00,2.00,4.4,6.80,0.496,chili with beans


In [88]:
demo_df[demo_df.demo_cat == "pasta enriched macaroni"]

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat


In [89]:
df[df.demo_cat == "pasta enriched macaroni"]

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat
3846,N11110849168,usda-ndb-import,"Pot-Ready Angel Hair Half Length Pasta, Enrich...",Kroger,kroger,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5688,N11150175951,usda-ndb-import,"No Boil Lasagna Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5691,N11150175999,usda-ndb-import,"Orzo Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5693,N11150176040,usda-ndb-import,"Spaghetti Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5694,N11150176095,usda-ndb-import,"Thin Spaghetti Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5695,N11150176125,usda-ndb-import,"Mostaccioli Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5696,N11150176132,usda-ndb-import,"Angel Hair Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5697,N11150176156,usda-ndb-import,"Medium Shells Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5698,N11150176163,usda-ndb-import,"Penne Rigate Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni
5699,N11150176170,usda-ndb-import,"Farfalle Pasta, Enriched Macaroni Product",Roundy's,roundy-s,56 g (2 oz),1494.0,1.79,0.0,73.21,3.57,3.6,12.50,0.000,pasta enriched macaroni


Now need to try some processing on product name as many will be too similar and uninformative without brand name.

In [90]:
def truncate_brand(s):
    if type(s) != str:
        return ""
    elif s.find(",") == -1:
        return s
    else:
        return s[:s.find(",")]

print(truncate_brand("Kroger, The Kroger Co."))
print(truncate_brand("Roundy's"))

Kroger
Roundy's


In [91]:
demo_df.dtypes

code                   object
creator                object
product_name           object
brands                 object
brands_tags            object
serving_size           object
energy_100g           float64
fat_100g              float64
cholesterol_100g      float64
carbohydrates_100g    float64
sugars_100g           float64
fiber_100g            float64
proteins_100g         float64
sodium_100g           float64
demo_cat               object
dtype: object

In [92]:
demo_df["short_brand"] = demo_df.brands.apply(truncate_brand)

/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [93]:
demo_df

,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat,short_brand
417,N901000017,usda-ndb-import,Yellow Corn Tortilla Chips,Guiltless Gourmet,guiltless-gourmet,28 g (1 oz),1795.0,10.71,0.000,78.57,0.00,7.1,7.14,0.643,yellow corn tortilla,Guiltless Gourmet
1366,N98601,usda-ndb-import,Chicken Noodle Soup,Trader Joe's,trader-joe-s,248 g (1 cup),151.0,0.40,0.008,5.65,0.40,0.4,2.42,0.294,chicken noodle soup,Trader Joe's
2267,N11110197900,usda-ndb-import,"Triple Layer Red Velvet Cake, Peppermint","Bakery Fresh Goodness, The Kroger Co.","bakery-fresh-goodness,the-kroger-co",85 g (1 SLICE),1971.0,17.65,0.024,77.65,65.88,0.0,1.18,0.353,red velvet cake,Bakery Fresh Goodness
2669,N11110577276,usda-ndb-import,Ice Cream Sandwich,The Kroger Co.,the-kroger-co,63 g (1 SANDWICH),1130.0,9.52,0.032,44.44,20.63,1.6,4.76,0.238,ice cream sandwich,The Kroger Co.
3172,N11110791559,usda-ndb-import,Crunchy Peanut Butter,Simple Truth Organic,simple-truth-organic,32 g (2 Tbsp),2615.0,50.00,0.000,18.75,6.25,9.4,25.00,0.375,crunchy peanut butter,Simple Truth Organic
3187,N11110796486,usda-ndb-import,"Kettle Cooked Potato Chips, Sea Salt & Cracked...",Kroger,kroger,28 g (1 oz),2243.0,32.14,0.000,53.57,0.00,3.6,7.14,0.393,cooked potato chips,Kroger
3427,N11110819376,usda-ndb-import,Dry Roasted Peanuts,Kroger,kroger,28 g (1 oz),2389.0,46.43,0.000,21.43,3.57,7.1,21.43,0.607,dry roasted peanuts,Kroger
3638,N11110836281,usda-ndb-import,53% Vegetable Oil Spread,P$$T... Big Savings... Pass It On,p-t-big-savings-pass-it-on,14 g (1 Tbsp),2092.0,50.00,0.000,0.00,0.00,0.0,0.00,0.643,vegetable oil spread,P$$T... Big Savings... Pass It On
3654,N11110837158,usda-ndb-import,98% Fat Free Cream Of Mushroom Condensed Soup,Kroger,kroger,125 g (0.5 cup),268.0,2.00,0.004,8.80,1.60,0.0,1.60,0.640,mushroom condensed soup,Kroger
3701,N11110839305,usda-ndb-import,Chili With Beans,Kroger,kroger,250 g (1 cup),502.0,4.00,0.010,14.00,2.00,4.4,6.80,0.496,chili with beans,Kroger


Now going to save original product name and replace with a combination of that plus the brand

In [94]:
demo_df["orig_product_name"] = demo_df.product_name

/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [95]:
demo_df["new_product_name"] = demo_df.short_brand + " " + demo_df.product_name

/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [96]:
demo_df.product_name = demo_df.new_product_name

/Users/seddont/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Need to add blanks for the other columns which only exist in the hand-curated demo data.

In [97]:
demo_df["hos"] = 0
demo_df["image_url"] = None

/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Now want to append the original demo data products and their hall of shame status so we always have them in too.

In [98]:
orig_demo = pd.read_csv(data_dir+"demo_food_data_latest.csv")

In [99]:
# Specify what columns we need to keep to match the regular dataframe above.

wanted_cols = ['code', 'creator', 'hos', 'image_url', 'product_name', 'brands', 'brands_tags', 'serving_size',
       'serving_size', 'energy_100g', 'fat_100g', 'cholesterol_100g',
       'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g',
        'demo_cat']

# Create a list of columns to drop to check it worked ok
drop_cols = [c for c in orig_demo.columns if c not in wanted_cols]
print(drop_cols)

['generic_name', 'quantity', 'categories', 'categories_tags', 'starch_100g', 'salt_100g', 'alcohol_100g', 'folates_100g', 'bicarbonate_100g', 'potassium_100g', 'chloride_100g', 'calcium_100g', 'iron_100g', 'fluoride_100g', 'iodine_100g', 'caffeine_100g', 'cocoa_100g']


In [100]:
# Drop unwanted columns in orig demo
orig_demo.drop(drop_cols, axis = 1, inplace = True)
orig_demo

,code,creator,hos,product_name,image_url,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat
0,N713733903387,usda-ndb-import,1,Low Sodium Applause Crackers,https://static.meijer.com/Media/007/13733/0071...,Meijer,meijer,16 g (16 g),2092,28.12,0.000,62.50,6.25,0.0,6.25,0.188,cracker
1,N876274001858,usda-ndb-import,0,"Snack Time, Stack Pack Salted Crackers",NaN,Global,global,28 g (28 g),2092,21.43,0.000,71.43,7.14,0.0,7.14,0.304,cracker
2,N888109050023,usda-ndb-import,1,"Donettes, Mini Donuts, Frosted",https://s-media-cache-ak0.pinimg.com/originals...,Hostess,hostess,85 g (3 oz),2017,30.59,0.012,48.24,30.59,1.2,3.53,0.306,donut
3,N72030015712,usda-ndb-import,0,Frosted Donuts,http://www.shugarysweets.com/wp-content/upload...,Entenmann's,entenmann-s,48 g (48 g),2004,31.25,0.021,47.92,27.08,2.1,4.17,0.375,donut
4,N24300806247,usda-ndb-import,1,Mini Donuts,http://bitemefoods.com.au/wp-content/uploads/2...,"Little Debbie, Mckee Foods Corporation","little-debbie,mckee-foods-corporation",122 g (8 DONUTS),1987,27.05,0.020,55.74,30.33,1.6,4.10,0.418,donut
5,N72030021645,usda-ndb-import,0,Pop'Ettes Powdered Donuts,https://jpg.seasonskosher.com/content/images/t...,"Entenmann's, Bimbo Bakeries Usa Inc.","entenmann-s,bimbo-bakeries-usa-inc",57 g (57 g),1983,28.07,0.026,52.63,26.32,1.8,3.51,0.368,donut
6,N11150146098,usda-ndb-import,1,"Kids Graham Crackers, Chocolate",http://www.candywarehouse.com/assets/item/regu...,Roundy's,roundy-s,30 g (19 COOKIES),1954,15.00,0.000,73.33,23.33,3.3,6.67,0.417,cracker
7,N76606720956,usda-ndb-import,0,"Savory Snack Cracker, Celery & Onion",http://www.baldersonvillagecheese.com/gourmet-...,Medford Farms,medford-farms,15 g (15 g),1954,26.67,0.000,60.00,0.00,0.0,6.67,1.000,cracker
8,N76265185905,usda-ndb-import,0,Plain Cake Mini Donuts,http://www.fridaycakenight.com/wp-content/uplo...,Maple Donuts Inc,maple-donuts-inc,45 g (2 DONUTS),1858,28.89,0.011,40.00,11.11,2.2,2.22,0.511,donut
9,N41224710146,usda-ndb-import,0,Roland Water Crackers,http://www.rolandfood.com/i/product_images/l/0...,Roland,roland,16 g (16 g),1833,6.25,0.000,75.00,0.00,6.2,6.25,0.750,cracker


In [101]:
# Drop unwanted columns in demo_df
# Create a list of columns to drop to check it worked ok
drop_cols = [c for c in demo_df.columns if c not in wanted_cols]
print(drop_cols)
demo_df.drop(drop_cols, axis = 1, inplace = True)
demo_df

['short_brand', 'orig_product_name', 'new_product_name']


/Users/seddont/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,code,creator,product_name,brands,brands_tags,serving_size,energy_100g,fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,demo_cat,hos,image_url
417,N901000017,usda-ndb-import,Guiltless Gourmet Yellow Corn Tortilla Chips,Guiltless Gourmet,guiltless-gourmet,28 g (1 oz),1795.0,10.71,0.000,78.57,0.00,7.1,7.14,0.643,yellow corn tortilla,0,None
1366,N98601,usda-ndb-import,Trader Joe's Chicken Noodle Soup,Trader Joe's,trader-joe-s,248 g (1 cup),151.0,0.40,0.008,5.65,0.40,0.4,2.42,0.294,chicken noodle soup,0,None
2267,N11110197900,usda-ndb-import,Bakery Fresh Goodness Triple Layer Red Velvet ...,"Bakery Fresh Goodness, The Kroger Co.","bakery-fresh-goodness,the-kroger-co",85 g (1 SLICE),1971.0,17.65,0.024,77.65,65.88,0.0,1.18,0.353,red velvet cake,0,None
2669,N11110577276,usda-ndb-import,The Kroger Co. Ice Cream Sandwich,The Kroger Co.,the-kroger-co,63 g (1 SANDWICH),1130.0,9.52,0.032,44.44,20.63,1.6,4.76,0.238,ice cream sandwich,0,None
3172,N11110791559,usda-ndb-import,Simple Truth Organic Crunchy Peanut Butter,Simple Truth Organic,simple-truth-organic,32 g (2 Tbsp),2615.0,50.00,0.000,18.75,6.25,9.4,25.00,0.375,crunchy peanut butter,0,None
3187,N11110796486,usda-ndb-import,"Kroger Kettle Cooked Potato Chips, Sea Salt & ...",Kroger,kroger,28 g (1 oz),2243.0,32.14,0.000,53.57,0.00,3.6,7.14,0.393,cooked potato chips,0,None
3427,N11110819376,usda-ndb-import,Kroger Dry Roasted Peanuts,Kroger,kroger,28 g (1 oz),2389.0,46.43,0.000,21.43,3.57,7.1,21.43,0.607,dry roasted peanuts,0,None
3638,N11110836281,usda-ndb-import,P$$T... Big Savings... Pass It On 53% Vegetabl...,P$$T... Big Savings... Pass It On,p-t-big-savings-pass-it-on,14 g (1 Tbsp),2092.0,50.00,0.000,0.00,0.00,0.0,0.00,0.643,vegetable oil spread,0,None
3654,N11110837158,usda-ndb-import,Kroger 98% Fat Free Cream Of Mushroom Condense...,Kroger,kroger,125 g (0.5 cup),268.0,2.00,0.004,8.80,1.60,0.0,1.60,0.640,mushroom condensed soup,0,None
3701,N11110839305,usda-ndb-import,Kroger Chili With Beans,Kroger,kroger,250 g (1 cup),502.0,4.00,0.010,14.00,2.00,4.4,6.80,0.496,chili with beans,0,None


In [102]:
missing_cols = [col for col in demo_df.columns if col not in orig_demo.columns]
print("Missing columns", missing_cols)

Missing columns []


Now want to append the original demo data to the other data

In [103]:
finished = demo_df.append(orig_demo)
finished

,brands,brands_tags,carbohydrates_100g,cholesterol_100g,code,creator,demo_cat,energy_100g,fat_100g,fiber_100g,hos,image_url,product_name,proteins_100g,serving_size,sodium_100g,sugars_100g
417,Guiltless Gourmet,guiltless-gourmet,78.57,0.000,N901000017,usda-ndb-import,yellow corn tortilla,1795.0,10.71,7.1,0,None,Guiltless Gourmet Yellow Corn Tortilla Chips,7.14,28 g (1 oz),0.643,0.00
1366,Trader Joe's,trader-joe-s,5.65,0.008,N98601,usda-ndb-import,chicken noodle soup,151.0,0.40,0.4,0,None,Trader Joe's Chicken Noodle Soup,2.42,248 g (1 cup),0.294,0.40
2267,"Bakery Fresh Goodness, The Kroger Co.","bakery-fresh-goodness,the-kroger-co",77.65,0.024,N11110197900,usda-ndb-import,red velvet cake,1971.0,17.65,0.0,0,None,Bakery Fresh Goodness Triple Layer Red Velvet ...,1.18,85 g (1 SLICE),0.353,65.88
2669,The Kroger Co.,the-kroger-co,44.44,0.032,N11110577276,usda-ndb-import,ice cream sandwich,1130.0,9.52,1.6,0,None,The Kroger Co. Ice Cream Sandwich,4.76,63 g (1 SANDWICH),0.238,20.63
3172,Simple Truth Organic,simple-truth-organic,18.75,0.000,N11110791559,usda-ndb-import,crunchy peanut butter,2615.0,50.00,9.4,0,None,Simple Truth Organic Crunchy Peanut Butter,25.00,32 g (2 Tbsp),0.375,6.25
3187,Kroger,kroger,53.57,0.000,N11110796486,usda-ndb-import,cooked potato chips,2243.0,32.14,3.6,0,None,"Kroger Kettle Cooked Potato Chips, Sea Salt & ...",7.14,28 g (1 oz),0.393,0.00
3427,Kroger,kroger,21.43,0.000,N11110819376,usda-ndb-import,dry roasted peanuts,2389.0,46.43,7.1,0,None,Kroger Dry Roasted Peanuts,21.43,28 g (1 oz),0.607,3.57
3638,P$$T... Big Savings... Pass It On,p-t-big-savings-pass-it-on,0.00,0.000,N11110836281,usda-ndb-import,vegetable oil spread,2092.0,50.00,0.0,0,None,P$$T... Big Savings... Pass It On 53% Vegetabl...,0.00,14 g (1 Tbsp),0.643,0.00
3654,Kroger,kroger,8.80,0.004,N11110837158,usda-ndb-import,mushroom condensed soup,268.0,2.00,0.0,0,None,Kroger 98% Fat Free Cream Of Mushroom Condense...,1.60,125 g (0.5 cup),0.640,1.60
3701,Kroger,kroger,14.00,0.010,N11110839305,usda-ndb-import,chili with beans,502.0,4.00,4.4,0,None,Kroger Chili With Beans,6.80,250 g (1 cup),0.496,2.00


Can now write this out to file.

In [104]:
# finished.to_csv(code_dir+"demo_food_data_final.csv", index = False)
finished.to_csv(code_dir+"demo_food_data_final2.csv", index = False)